### Archived Statements - part 3

Target:

 - us-east = 809,993,275
 - us-west = 809,993,275
 - archives = 3,239,979,516+ 2,159,986,344 = 5,399,965,860

 
Process:

 - bulk import via stream consumer of import/statements_archive3
 - use externalScroller to bulk index at the same time

During the last // import we saw that the throughput ended up dropping to 20Kdocs/s, so the gain of having 2 injectors running concurrently does not seem that useful.
Let's use the remaining MongoDB throughput to do the indexing and avoid having to redo a full scroll+reindex.

#### Run the imports

In [3]:
!import.sh -o import -t 24 -l import/statements_archive3 -r archives -z 500 -b / -a -w 200000 -bulk -useScroller > logs/import-archives-step3.log

Exception in thread "main" org.nuxeo.client.spi.NuxeoClientException: Error during call on url=http://127.0.0.1:8080/nuxeo/site/api/v1/automation/StreamImporter.runDocumentConsumersEx/@async/eadf8ebe-9200-4f44-b4eb-4ddb17c04568/status
	at org.nuxeo.client.NuxeoClient.request(NuxeoClient.java:261)
	at org.nuxeo.client.NuxeoClient.get(NuxeoClient.java:233)
	at org.nuxeo.data.gen.cli.NuxeoClientHelper.waitForResult(NuxeoClientHelper.java:105)
	at org.nuxeo.data.gen.cli.RestCli.main(RestCli.java:255)
Caused by: java.net.SocketException: Connection reset
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:186)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:140)
	at okio.InputStreamSource.read(Okio.kt:102)
	at okio.AsyncTimeout$source$1.read(AsyncTimeout.kt:159)
	at okio.RealBufferedSource.indexOf(RealBufferedSource.kt:349)
	at okio.RealBufferedSource.readUtf8LineStrict(RealBufferedSource.kt:222)
	at okhttp3.internal.http1.Http1Codec.readHeaderLine(Http

Started BulkIndex using ExternalScroller: ee492d2f-112a-4f84-90ef-e63ae146c886

In [5]:
# check status
! cid="ee492d2f-112a-4f84-90ef-e63ae146c886"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:archives' \
  -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid

{"entity-type":"bulkStatus","commandId":"ee492d2f-112a-4f84-90ef-e63ae146c886","state":"SCROLLING_RUNNING","processed":434609856,"error":false,"errorCount":0,"total":0,"action":"index","username":"nco-admin","submitted":"2020-09-20T13:13:27.362Z","scrollStart":"2020-09-20T13:13:27.384Z","scrollEnd":null,"processingStart":null,"processingEnd":null,"completed":null,"processingMillis":0}

The first run failed because of OOM: see https://jira.nuxeo.com/browse/SUPINT-1789

In [6]:
# cancel previously running BAF
! cid="ee492d2f-112a-4f84-90ef-e63ae146c886"; \
  INJECTOR="http://127.0.0.1:8080"; \
  curl -H 'Content-Type:application/json+nxrequest' -H 'X-NXRepository:archives' \
  -X PUT -u $NXUSER:$NXPWD $INJECTOR/nuxeo/api/v1/bulk/$cid/abort

{"entity-type":"bulkStatus","commandId":"ee492d2f-112a-4f84-90ef-e63ae146c886","state":"ABORTED","processed":434609856,"error":false,"errorCount":0,"total":0,"action":"index","username":"nco-admin","submitted":"2020-09-20T13:13:27.362Z","scrollStart":"2020-09-20T13:13:27.384Z","scrollEnd":null,"processingStart":null,"processingEnd":null,"completed":null,"processingMillis":0}

In [ ]:
!import.sh -o import -t 24 -l import/statements_archive3 -r archives -z 500 -b / -a -w 200000 -bulk -useScroller > logs/import-archives-step3-2.log